In [1]:
import os,sys,time,datetime
import os.path as osp
import numpy as np
from IPython import embed

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler

In [3]:
from model.ReIDNet import ReIDNet
from dataset_manager import Market1501
from dataset_loader import ImageDataset
from TriHard_Loss import TripletLoss
from sampler import RandomIdentitySampler
import transform as T

In [4]:
#设定输入参数
width=64                    #图片宽度
height=128                 #图片高度
train_batch_size=32  #训练批量
test_batch_size=32  #测试批量
train_lr=0.01                #学习率
start_epoch=0           #开始训练的批次
end_epoch=1                 #结束训练的批次
dy_step_size=800      #动态学习率变化步长
dy_step_gamma=0.9  #动态学习率变化倍数
evaluate=False           #是否测试
margin=0.3                 #TripletHard Loss计算的margin参数
num_instances=4        #每个ID图片数，一定要能被batch_size整除
max_acc=-1
best_model_path='./model/param/trihard_net_params_best.pth'#最优模型保存地址
final_model_path='./model/param/trihard_net_params_final.pth'#最终模型保存地址

In [5]:
def main():
    #数据集加载
    dataset=Market1501()
    #训练数据处理器
    transform_train=T.Compose([
        T.Random2DTransform(height,width),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
    )
    #测试数据处理器
    transform_test=T.Compose([
        T.Resize((height,width)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
    )
    #train数据集吞吐器
    train_data_loader=DataLoader(
        ImageDataset(dataset.train, transform=transform_train),
        sampler=RandomIdentitySampler(dataset.train, num_instances=num_instances),
        batch_size=train_batch_size,
        drop_last=True,
    )
    print("train_data_loader inited")
    #query数据集吞吐器
    query_data_loader=DataLoader(
        ImageDataset(dataset.query, transform=transform_test),
        batch_size=test_batch_size,
        shuffle=False,
        drop_last=True,
    )
    print("query_data_loader inited")
    #gallery数据集吞吐器
    gallery_data_loader=DataLoader(
        ImageDataset(dataset.gallery, transform=transform_test),
        batch_size=test_batch_size,
        shuffle=False,
        drop_last=True,
    )
    print("gallery_data_loader inited\n")
    
    #加载模型
    model=ReIDNet(num_classes=751,loss={'softmax, metric'})#也可以只用softmax或只用metric
    print("=>ReIDNet loaded")
    print("Model size: {:.5f}M\n".format(sum(p.numel() for p in model.parameters())/1000000.0))
    
    #损失函数
    criterion_class=nn.CrossEntropyLoss()
    criterion_metric=TripletLoss(margin=margin)
    
    #优化器
    optimizer=torch.optim.SGD(model.parameters(),lr=train_lr,weight_decay=5e-04)
    
    #动态学习率
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=dy_step_gamma, patience=10, min_lr=0.0001)
    
    if evaluate:
        test(model,query_data_loader,gallery_data_loader)
        return 0
    print('————model start training————\n')
    bt=time.time()
    for epoch in range(start_epoch,end_epoch):
        model.train(True)
        train(epoch,model,criterion_class, criterion_metric, optimizer,scheduler,train_data_loader)
    et=time.time()
    print('**模型训练结束, 保存最终参数到{}**\n'.format(final_model_path))
    torch.save(model.state_dict(), final_model_path)
    print('————训练总用时{:.2f}小时————'.format((et-bt)/3600.0))

In [6]:
def train(epoch, model, criterion_class, criterion_metric, optimizer, scheduler, data_loader):
    global max_acc
    for batch_idx, (imgs, pids, cids) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs, features=model(imgs)#当用了两种损失时，就有两个输出
        xent_loss=criterion_class(outputs,pids)
        metric_loss=criterion_metric(features, pids)
        loss=xent_loss+metric_loss
        loss.backward()
        scheduler.step(loss)
        optimizer.step()
        pred = torch.argmax(outputs, 1)
        current_acc=100*(pred == pids).sum().float()/len(pids)
        if current_acc>max_acc:
            max_acc=current_acc
            print('**最高准确度更新为{}%，保存此模型到{}**\n'.format(max_acc,best_model_path))
            torch.save(model.state_dict(), best_model_path)
        if batch_idx%20==0:
            print('————————————————————————————————')
            pred = torch.argmax(outputs, 1)
            print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch + 1, batch_idx, loss.data))
            print('Current Accuracy: {:.2f}%'.format(100*(pred == pids).sum().float()/len(pids)))     
            print('————————————————————————————————\n')
    return 0

In [7]:
main()

=> Market1501 loaded
------------------------------------------------------------------------
  subset: train  	| num_id:   751  	|  num_imgs:   12936  
  subset: query  	| num_id:   750  	|  num_imgs:    3368  
  subset: gallery 	| num_id:   751  	|  num_imgs:   19732  
------------------------------------------------------------------------
  total 			| num_id:  1501  	|  num_imgs:   16304  
------------------------------------------------------------------------


TypeError: __init__() got an unexpected keyword argument 'transform'

In [ ]:
"""
model=ReIDNet(num_classes=751,loss='softmax')
model.load_state_dict(torch.load('./model/net_params.pkl'))
print(model)

dataset=Market1501()
#训练数据处理器
transform_test=T.Compose([
        T.Resize((height,width)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]
)
#train数据集吞吐器
train_data_loader=DataLoader(
    ImageDataset(dataset.train, transform=transform_test),
    batch_size=train_batch_size,
    drop_last=True,
)
correct=0
total=0
for batch_idx, (imgs, pids, cids) in enumerate(train_data_loader):
    outputs=model(imgs)
    pred = torch.argmax(outputs, 1)
    correct += (pred == pids).sum().float()
    total += len(pids)
acc_str = 'Accuracy: %f'%((correct/total))
print(acc_str)
"""